# Model 2: Stacking

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns

from imblearn.over_sampling import RandomOverSampler

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, roc_auc_score, roc_curve, classification_report, f1_score, fbeta_score, make_scorer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.compose import make_column_transformer
from imblearn.combine import SMOTEENN
from imblearn.pipeline import Pipeline as imb_pipeline

from xgboost import XGBClassifier



pd.set_option('display.float_format', lambda x: '%.2f' % x)
RSEED = 42

In [2]:
# df = pd.read_csv('data/cleaned/train.csv')
# df_test = pd.read_csv('data/cleaned/test.csv')
df = pd.read_csv('data/cleaned/train_extended.csv')
df_test = pd.read_csv('data/cleaned/test_extended.csv')
df.head()


,district,client_id,client_catg,region,reading_remarque_6,reading_remarque_7,reading_remarque_8,reading_remarque_9,reading_remarque_413,counter_statue_0,...,counter_coefficient_30,counter_coefficient_33,counter_coefficient_40,counter_coefficient_50,counter_code_count_unique,counter_number_count_unique,counter_type_count_unique,months_number,consommation_sum,target
0,60,train_Client_29310,11,101,18,0,6,34,0,56,...,0,0,0,0,2,3,2,3.93,734.81,0.00
1,63,train_Client_128477,11,312,1,0,2,1,0,4,...,0,0,0,0,1,1,1,3.50,325.75,0.00
2,63,train_Client_56966,11,311,40,0,15,23,0,78,...,0,0,0,0,2,2,2,4.21,537.35,0.00
3,69,train_Client_23694,11,104,0,0,3,17,0,20,...,0,0,0,0,2,2,2,4.00,762.45,0.00
4,62,train_Client_12064,11,309,2,0,0,24,0,26,...,0,0,0,0,2,2,2,4.00,375.73,0.00


## Split Target, Drop ID

In [3]:
y_train = df['target']
X_train = df.drop(['target', 'client_id'], axis=1)

In [4]:
#Split target from test data
y_test = df_test['target']
X_test = df_test.drop(['target', 'client_id'], axis=1)

## Feature Engineering

#### Replace 0

In [ ]:
value = 'consommation_sum'
# value = 'months_number'
zeros_train = X_train[X_train[value]==0].shape
print('zeros in trainset:', zeros_train[0])
print(X_train[value].isnull().sum())

zeros_test = X_test[X_test[value]==0].shape
print('zeros in testset:', zeros_test[0])
print(X_test[value].isnull().sum())

In [ ]:
sns.histplot(X_train['consommation_sum'])

In [ ]:
sns.histplot(X_train['consommation_per_month'])

In [ ]:
sns.histplot(X_train['months_number'])

In [ ]:
# to impute the 0 in the consommation_sum, we need to replace the 0 with nan
X_train['consommation_sum'].replace(0, np.nan, inplace=True)
X_test['consommation_sum'].replace(0, np.nan, inplace=True)

#### ColumnTransformer

In [5]:
# numerical features
num_features = [ 'consommation_sum', 'months_number']
num_transformer = make_pipeline(
        MinMaxScaler(), # no gaussian distribution? 
    )

In [6]:
preprocessor = make_column_transformer(
        (num_transformer, num_features)
)

## Imbalanced Data

### Random Oversampler

In [ ]:
# handling the imbalanced
ros = RandomOverSampler(random_state=RSEED)
X_train, y_train = ros.fit_resample(X_train, y_train)

sns.countplot(x=y_train)

### Undersampling

In [7]:
# # SMOTE + ENN (Noise Reduction)
smote_enn = SMOTEENN(random_state=42)

X_train, y_train = smote_enn.fit_resample(X_train, y_train)


In [ ]:
sns.countplot(x=y_train)

## Baseline model 1: Decision Tree


In [8]:
#baseline model 1: Decision tree
baseline_tree = DecisionTreeClassifier(random_state=RSEED, max_depth=7)
baseline_tree.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=7, random_state=42)

In [9]:
print(f'Decision tree has {baseline_tree.tree_.node_count} nodes with maximum depth {baseline_tree.tree_.max_depth}.')
print(f'On average there are ca. {X_train.shape[0]/baseline_tree.tree_.node_count:.1f} data points in each leaf.')

Decision tree has 243 nodes with maximum depth 7.
On average there are ca. 655.9 data points in each leaf.


In [ ]:
# fig = plt.figure(figsize=(25,10))
# dectree_plot = plot_tree(baseline_tree, filled=True)

In [11]:
# Make probability predictions for X_train
train_probs1 = baseline_tree.predict_proba(X_train)[:, 1]
train_predictions1 = baseline_tree.predict(X_train)

In [12]:
print(f'Train ROC AUC Score: {roc_auc_score(y_train, train_probs1)}')

Train ROC AUC Score: 0.8583759731991472


In [13]:
print(confusion_matrix(y_train, train_predictions1))
print(classification_report(y_train, train_predictions1))

[[46790 21740]
 [12381 78464]]
              precision    recall  f1-score   support

         0.0       0.79      0.68      0.73     68530
         1.0       0.78      0.86      0.82     90845

    accuracy                           0.79    159375
   macro avg       0.79      0.77      0.78    159375
weighted avg       0.79      0.79      0.78    159375



In [14]:
# Make probability predictions test data
test_probs1 = baseline_tree.predict_proba(X_test)[:, 1]
test_predictions1 = baseline_tree.predict(X_test)

In [15]:
print(f'Test ROC AUC Score: {roc_auc_score(y_test, test_probs1)}')

Test ROC AUC Score: 0.7576324423974478


In [16]:
print(confusion_matrix(y_test, test_predictions1))
print(classification_report(y_test, test_predictions1))

[[19137 12831]
 [  427  1464]]
              precision    recall  f1-score   support

         0.0       0.98      0.60      0.74     31968
         1.0       0.10      0.77      0.18      1891

    accuracy                           0.61     33859
   macro avg       0.54      0.69      0.46     33859
weighted avg       0.93      0.61      0.71     33859



## Baseline model 2: Logistic Regression

In [ ]:
# baseline 2: logistic Regression

baseline_log_reg = LogisticRegression(max_iter=1000, solver='lbfgs', n_jobs=1)
baseline_log_reg.fit(X_train, y_train)

# Make probability predictions for X_train
train_probs2 = baseline_log_reg.predict_proba(X_train)[:, 1]
train_predictions2 = baseline_log_reg.predict(X_train)

In [ ]:
# Results X_train prediction:
print(f'Train ROC AUC Score: {roc_auc_score(y_train, train_probs2)}')
print(confusion_matrix(y_train, train_predictions2))
print(classification_report(y_train, train_predictions2))

In [ ]:
# Make probability predictions for X_test
test_probs2 = baseline_log_reg.predict_proba(X_test)[:, 1]
test_predictions2 = baseline_log_reg.predict(X_test)

# Results X_test prediction:
print(f'Test ROC AUC Score: {roc_auc_score(y_test, test_probs2)}')
print(confusion_matrix(y_test, test_predictions2))
print(classification_report(y_test, test_predictions2))

## HyperparameterSearch & Scoring


In [10]:
scoring = make_scorer(fbeta_score, beta=2)
# scoring = 'roc_auc'
cv = 5 #RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

## Model 2: Stacking

In [11]:
# Stack the models Decision Tree, KNN and SGDClassifier
estimators = [
    ('dt', DecisionTreeClassifier(random_state = RSEED)),
    ('knn', KNeighborsClassifier()),
    ('class', SGDClassifier())
]

In [12]:
pipe_st = Pipeline([
    ('preprocessor', preprocessor),
    ('st', StackingClassifier(estimators = estimators, final_estimator = LogisticRegression()))]
)

In [13]:
# don't forget to add the name of the model from the pipelin in front of each hyperparameter!
param_st = {'st__dt__max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'st__dt__max_leaf_nodes': [None] + list(np.arange(10, 51).astype(int)),
    'st__dt__min_samples_split': [2, 5, 10],
    'st__dt__max_depth': [None] + list(np.arange(3, 21).astype(int)),
    'st__knn__n_neighbors' : np.arange(2,50), #this actually defines the model you use
    'st__knn__weights' : ['uniform', 'distance'],
    'st__knn__p' : [1, 2, 3],
    'st__knn__algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'st__class__penalty':('l1','l2'),
    'st__class__alpha': [0.001, 0.01, 0.1, 1, 10],
    'st__class__learning_rate': ['optimal', 'constant', 'invscaling'],
    'st__class__loss': ['log_loss', 'hinge', 'huber'],
    'st__class__max_iter' : [1000, 500],
    'st__class__eta0': [0.001, 0.01]
    }

randomsearch_st = RandomizedSearchCV(pipe_st, param_distributions=param_st, cv=cv, scoring=scoring, n_iter=10,
                           verbose=5, n_jobs=-1) #evt. add error_score='raise'

In [14]:
# randomsearch_st.fit(X_train, y_train)
randomsearch_st.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END st__class__alpha=0.001, st__class__eta0=0.001, st__class__learning_rate=invscaling, st__class__loss=huber, st__class__max_iter=1000, st__class__penalty=l2, st__dt__max_depth=4, st__dt__max_features=0.6, st__dt__max_leaf_nodes=27, st__dt__min_samples_split=5, st__knn__algorithm=ball_tree, st__knn__n_neighbors=15, st__knn__p=3, st__knn__weights=distance;, score=0.804 total time=  30.4s
[CV 2/5] END st__class__alpha=0.001, st__class__eta0=0.001, st__class__learning_rate=invscaling, st__class__loss=huber, st__class__max_iter=1000, st__class__penalty=l2, st__dt__max_depth=4, st__dt__max_features=0.6, st__dt__max_leaf_nodes=27, st__dt__min_samples_split=5, st__knn__algorithm=ball_tree, st__knn__n_neighbors=15, st__knn__p=3, st__knn__weights=distance;, score=0.844 total time=  31.2s
[CV 3/5] END st__class__alpha=0.001, st__class__eta0=0.001, st__class__learning_rate=invscaling, st__class__loss=huber, st__class__max_iter

/Users/katharinabaumgartner/Documents/NeueFische/scripts/24_ensembleMethods/ds-ensemble-methods/.venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/katharinabaumgartner/Documents/NeueFische/scripts/24_ensembleMethods/ds-ensemble-methods/.venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/katharinabaumgartner/Documents/NeueFische/scripts/24_ensembleMethods/ds-ensemble-methods/.venv/lib/python3.11/site-packag

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('pipeline',
                                                                               Pipeline(steps=[('minmaxscaler',
                                                                                                MinMaxScaler())]),
                                                                               ['consommation_sum',
                                                                                'months_number'])])),
                                             ('st',
                                              StackingClassifier(estimators=[('dt',
                                                                              DecisionTreeClassifier(random_state=42)),
                                                                             ('knn',
                                                                              KNeighborsClassifier()),
                                                                             ('class',
                                                                              SGDClassifier())],
                                                                 final_e...
                                        'st__knn__algorithm': ['ball_tree',
                                                               'kd_tree',
                                                               'brute'],
                                        'st__knn__n_neighbors': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
                                        'st__knn__p': [1, 2, 3],
                                        'st__knn__weights': ['uniform',
                                                             'distance']},
                   scoring=make_scorer(fbeta_score, response_method='predict', beta=2),
                   verbose=5)

In [15]:
# Show best parameters
print('Best score:\n{:.2f}'.format(randomsearch_st.best_score_))
print("Best parameters:\n{}".format(randomsearch_st.best_params_))

Best score:
0.85
Best parameters:
{'st__knn__weights': 'uniform', 'st__knn__p': 3, 'st__knn__n_neighbors': 27, 'st__knn__algorithm': 'kd_tree', 'st__dt__min_samples_split': 10, 'st__dt__max_leaf_nodes': 33, 'st__dt__max_features': 0.7, 'st__dt__max_depth': 9, 'st__class__penalty': 'l1', 'st__class__max_iter': 500, 'st__class__loss': 'huber', 'st__class__learning_rate': 'invscaling', 'st__class__eta0': 0.01, 'st__class__alpha': 10}


In [16]:
# Save best model (including fitted preprocessing steps) as best_model_st
best_model_st = randomsearch_st.best_estimator_
best_model_st

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['consommation_sum',
                                                   'months_number'])])),
                ('st',
                 StackingClassifier(estimators=[('dt',
                                                 DecisionTreeClassifier(max_depth=9,
                                                                        max_features=0.7,
                                                                        max_leaf_nodes=33,
                                                                        min_samples_split=10,
                                                                        random_state=42)),
                                                ('knn',
                                                 KNeighborsClassifier(algorithm='kd_tree',
                                                                      n_neighbors=27,
                                                                      p=3)),
                                                ('class',
                                                 SGDClassifier(alpha=10,
                                                               eta0=0.01,
                                                               learning_rate='invscaling',
                                                               loss='huber',
                                                               max_iter=500,
                                                               penalty='l1'))],
                                    final_estimator=LogisticRegression()))])

In [17]:
# best_model_st.score(X_test, y_test)

In [18]:
test_predictions_1 = best_model_st.predict(X_test)

In [19]:
test_probs_1 = best_model_st.predict_proba(X_test)[:, 1]


In [20]:
print(f'Test ROC AUC Score: {roc_auc_score(y_test, test_probs_1)}')
print(confusion_matrix(y_test, test_predictions_1))
print(classification_report(y_test, test_predictions_1))

Test ROC AUC Score: 0.6166360206054813
[[13992 17976]
 [  503  1388]]
              precision    recall  f1-score   support

         0.0       0.97      0.44      0.60     31968
         1.0       0.07      0.73      0.13      1891

    accuracy                           0.45     33859
   macro avg       0.52      0.59      0.37     33859
weighted avg       0.92      0.45      0.58     33859



In [21]:
# Make predictions for X_train
train_probs_2 = best_model_st.predict_proba(X_train)[:, 1]
train_predictions_2 = best_model_st.predict(X_train)

In [22]:
# Results X_train prediction:
print(f'Train ROC AUC Score: {roc_auc_score(y_train, train_probs_2)}')
print(confusion_matrix(y_train, train_predictions_2))
print(classification_report(y_train, train_predictions_2))

Train ROC AUC Score: 0.7906381753550378
[[35513 33017]
 [10033 80812]]
              precision    recall  f1-score   support

         0.0       0.78      0.52      0.62     68530
         1.0       0.71      0.89      0.79     90845

    accuracy                           0.73    159375
   macro avg       0.74      0.70      0.71    159375
weighted avg       0.74      0.73      0.72    159375



In [23]:
# Compute FPR and TPR for the model to plot the roc_curve
fpr_values_st, tpr_values_st, _ = roc_curve(y_test, test_probs_1)  # Probability of class 1
np.save("fpr_values_st.npy", fpr_values_st)
np.save("tpr_values_st.npy", tpr_values_st)

In [24]:
# Making a list with the results for plotting
st_metrics = np.array([accuracy_score(y_test, test_predictions_1), precision_score(y_test, test_predictions_1), recall_score(y_test, test_predictions_1), f1_score(y_test, test_predictions_1)]).round(2)
st_metrics

array([0.45, 0.07, 0.73, 0.13])

## Evaluate the models

In [ ]:
#NOCH ANZUPASSEN!


def evaluate_model(predictions, probs, train_predictions, train_probs):
    """Compare machine learning model to baseline performance.
    Computes statistics and shows ROC curve."""
    
   baseline_tree = {}
    
   baseline_tree['recall'] = recall_score(y_test, [1 for _ in range(len(y_test))])
   baseline_tree['precision'] = precision_score(y_test, [1 for _ in range(len(y_test))])
   baseline_tree['roc'] = roc_auc_score(y_test, y_probs)

   baseline_log_reg = {}
    
   baseline_log_reg['recall'] = recall_score(y_test, [1 for _ in range(len(y_test))])
   baseline_log_reg['precision'] = precision_score(y_test, [1 for _ in range(len(y_test))])
   baseline_log_reg['roc'] = roc_auc_score(y_test, y_probs)
    
    results = {}
    
    results['recall'] = recall_score(y_test, predictions)
    results['precision'] = precision_score(y_test, predictions)
    results['roc'] = roc_auc_score(y_test, probs)
    
    train_results = {}
    train_results['recall'] = recall_score(train_labels, train_predictions)
    train_results['precision'] = precision_score(train_labels, train_predictions)
    train_results['roc'] = roc_auc_score(train_labels, train_probs)
    
    for metric in ['recall', 'precision', 'roc']:
        print(f'{metric.capitalize()} Baseline: {round(baseline[metric], 2)} Test: {round(results[metric], 2)} Train: {round(train_results[metric], 2)}')
    
    # Calculate false positive rates and true positive rates
    base_fpr, base_tpr, _ = roc_curve(y_test, [1 for _ in range(len(y_test))])
    model_fpr, model_tpr, _ = roc_curve(y_test, probs)

    plt.figure(figsize = (8, 6))
    plt.rcParams['font.size'] = 16
    
    # Plot both curves
    plt.plot(base_fpr, base_tpr, 'b', label = 'baseline')
    plt.plot(model_fpr, model_tpr, 'r', label = 'model')
    plt.legend();
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.title('ROC Curves');